In [368]:
import math

preprocessed_dataset_folder = "../preprocessed_dataset"

import pandas as pd
from lenskit import topn
from sklearn.metrics import ndcg_score
from statistics import mean, variance
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
pd.options.mode.chained_assignment = None  # default='warn'

I first generated random groups of 3 and generated recommendations, aggregated them and evaluated using ndcg. Then I made a method at the very end that does all of it in a single method for ease of use.

In [338]:
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv") 
display(ratings_df.head(10))

,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,70,3.0,964982400
4,1,101,5.0,964980868
5,1,110,4.0,964982176
6,1,151,5.0,964984041
7,1,157,5.0,964984100
8,1,163,5.0,964983650
9,1,216,5.0,964981208


In [339]:
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv", index_col="item")
display(movies_df.head(10))

,title,year,genres,plot
item,,,,
1,toy story,1995,Adventure|Animation|Children|Comedy|Fantasy,In a world where toys are living things who pr...
2,jumanji,1995,Adventure|Children|Fantasy,"In 1869, near Brantford, New Hampshire, two br..."
3,grumpier old men,1995,Comedy|Romance,The feud between Max (Walter Matthau) and John...
4,waiting to exhale,1995,Comedy|Drama|Romance,"""Friends are the People who let you be yoursel..."
5,father of the bride part ii,1995,Comedy,The film begins five years after the events of...
6,heat,1995,Action|Crime|Thriller,"Neil McCauley, a career criminal, hires Waingr..."
7,sabrina,1995,Comedy|Romance,Sabrina Fairchild is the young daughter of the...
8,tom and huck,1995,Adventure|Children,The movie opens with Injun Joe (Eric Schweig) ...
9,sudden death,1995,Action,Darren McCord (Jean-Claude Van Damme) is a Fre...


In [340]:
users_ratings = ratings_df.groupby(['user']).count()

groupsize = 3

selected = users_ratings['rating'] > 200
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample(groupsize) # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
selected_users = select_column_df.values # iloc select by index, since our dataframe only has one row we read it from the index 0
print("Selected users: " + str(selected_users))

Selected users: [ 45 561 357]


In [341]:
ratings_df

,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,70,3.0,964982400
4,1,101,5.0,964980868
...,...,...,...,...
64433,610,163937,3.5,1493848789
64434,610,166534,4.0,1493848402
64435,610,168250,5.0,1494273047
64436,610,168252,5.0,1493846352


In [342]:


train_df, test_df = train_test_split(ratings_df, test_size=0.2)

user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(train_df)

test_df['predicted_rating'] = recsys.predict(test_df)

test_df['relevant'] = test_df['rating'].apply(lambda x: 1 if x>3 else 0)
test_df['predicted_relevant'] = test_df['predicted_rating'].apply(lambda x: 1 if x>3 else 0)



In [343]:


train_df, test_df = train_test_split(ratings_df, test_size=0.2)

user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(train_df)

In [344]:
selected_users

array([ 45, 561, 357], dtype=int64)

In [345]:
glist = []
for u in selected_users:
    selected_movies = recsys.recommend(u, 10)
    # selected_movies["user"] = u
    glist.append(selected_movies)



In [346]:
groupdf = pd.concat(glist)

In [347]:
groupdf

,item,score
0,3030,5.463694
1,3334,5.131901
2,7706,5.131306
3,2135,4.991196
4,1248,4.955991
5,951,4.955703
6,4117,4.923618
7,51931,4.889465
8,3451,4.864584
9,3201,4.820582


In [348]:
groupdf.drop_duplicates(subset=["item"], inplace=True)

In [349]:
lst = []
for u in selected_users:
    print(u)
    g = groupdf.copy()
    g["user"] = u
    g["pred"] = recsys.predict_for_user(u, g["item"]).values
    lst.append(g)

45
561
357


In [350]:
final = pd.concat(lst)

In [351]:
final.sort_values(by=["pred"], inplace=True, ascending=False)
final

,item,score,user,pred
1,3814,4.903489,357,5.472889
0,3030,5.463694,45,5.463694
1,128520,5.260225,357,5.260225
2,5292,5.202833,357,5.202833
0,3030,5.463694,357,5.187576
...,...,...,...,...
9,53129,5.020185,561,NaN
2,7706,5.131306,357,NaN
6,4117,4.923618,357,NaN
6,2936,4.608429,357,NaN


In [352]:
gb = final.groupby("item", as_index=False)["pred"].mean()
gb

,item,pred
0,947,4.478695
1,951,4.815858
2,1248,4.710030
3,2135,4.197299
4,2936,4.499606
5,3030,5.209178
6,3201,4.743641
7,3334,4.717530
8,3451,4.730149
9,3494,4.637725


In [353]:
# 5 recommendations
final_recommendations = gb.sort_values(by=["pred"], ascending=False).head(5)
final_recommendations

,item,pred
5,3030,5.209178
10,3814,5.188189
18,26810,5.129983
20,53129,5.020185
17,7706,4.960511


These are the final recommendations in order

In [354]:
final_recommendations = final_recommendations.join(movies_df['title'], on='item')
final_recommendations

,item,pred,title
5,3030,5.209178,yojimbo
10,3814,5.188189,love and death
18,26810,5.129983,bad boy bubby
20,53129,5.020185,mr. brooks
17,7706,4.960511,animal crackers


In [355]:
evaluate = final_recommendations.copy()
for u in selected_users:
    evaluate[u] = recsys.predict_for_user(u, final_recommendations["item"]).values
evaluate

,item,pred,title,45,561,357
5,3030,5.209178,yojimbo,5.463694,4.976264,5.187576
10,3814,5.188189,love and death,NaN,4.903489,5.472889
18,26810,5.129983,bad boy bubby,NaN,NaN,5.129983
20,53129,5.020185,mr. brooks,NaN,NaN,5.020185
17,7706,4.960511,animal crackers,5.131306,4.789715,NaN


In [356]:
# if scores cannot be predicted replace with 0
evaluate.fillna(0, inplace=True)
#evaluate.set_index("item", inplace=True)

In [357]:
recommended_ordering = pd.DataFrame(evaluate[["item","pred"]])
recommended_ordering

,item,pred
5,3030,5.209178
10,3814,5.188189
18,26810,5.129983
20,53129,5.020185
17,7706,4.960511


In [358]:
ndcg_lst = []

for u in selected_users:
    evaluate.sort_values(by=[u], ascending=False, inplace=True)
    ev = pd.DataFrame(evaluate[["item", u]])
    #ev["rank" + str(u)] = ev[u].rank(ascending=False)
    #print(ev)

    val = ndcg_score([ev.item], [recommended_ordering.item])

    ndcg_lst.append(val)

ndcg_lst

[0.7856897309644435, 0.7958496215602588, 0.9995521192949273]

In [359]:
#take the mean of the ndcg scores
mean(ndcg_lst)

0.8603638239398765

In [360]:
def group_pipeline(groupsize):

    selected = users_ratings['rating'] > 200
    selected_users = users_ratings.loc[selected]
    random_selected = selected_users.sample(groupsize) # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
    select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
    selected_users = select_column_df.values # iloc select by index, since our dataframe only has one row we read it from the index 0
    print("Selected users: " + str(selected_users))

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)

    test_df['predicted_rating'] = recsys.predict(test_df)

    test_df['relevant'] = test_df['rating'].apply(lambda x: 1 if x > 3 else 0)
    test_df['predicted_relevant'] = test_df['predicted_rating'].apply(lambda x: 1 if x > 3 else 0)

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)

    glist = []
    for u in selected_users:
        selected_movies = recsys.recommend(u, 10)
        # selected_movies["user"] = u
        glist.append(selected_movies)

    groupdf = pd.concat(glist)

    groupdf.drop_duplicates(subset=["item"], inplace=True)
    lst = []
    for u in selected_users:

        g = groupdf.copy()
        g["user"] = u
        g["pred"] = recsys.predict_for_user(u, g["item"]).values
        lst.append(g)

    final = pd.concat(lst)
    final.sort_values(by=["pred"], inplace=True, ascending=False)

    gb = final.groupby("item", as_index=False)["pred"].mean()

    # 5 recommendations
    final_recommendations = gb.sort_values(by=["pred"], ascending=False).head(5)

    final_recommendations = final_recommendations.join(movies_df['title'], on='item')

    evaluate = final_recommendations.copy()
    for u in selected_users:
        evaluate[u] = recsys.predict_for_user(u, final_recommendations["item"]).values

    evaluate.fillna(0, inplace=True)
    #evaluate.set_index("item", inplace=True)
    recommended_ordering = pd.DataFrame(evaluate[["item", "pred"]])

    ndcg_lst = []

    for u in selected_users:
        evaluate.sort_values(by=[u], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluate[["item", u]])
        #ev["rank" + str(u)] = ev[u].rank(ascending=False)
        #print(ev)

        val = ndcg_score([ev.item], [recommended_ordering.item])

        ndcg_lst.append(val)


    return mean(ndcg_lst)

In [361]:
gsize2 = []
for i in range(1000):
    gsize2.append(group_pipeline(2))

mean(gsize2)

Selected users: [219 580]
Selected users: [140 339]
Selected users: [608 103]
Selected users: [ 64 596]
Selected users: [318 597]
Selected users: [567  28]
Selected users: [305 480]
Selected users: [ 42 105]
Selected users: [51 19]
Selected users: [202 480]
Selected users: [305 275]
Selected users: [357  64]
Selected users: [156  66]
Selected users: [597 103]
Selected users: [292 534]
Selected users: [ 57 525]
Selected users: [275 156]
Selected users: [156 357]
Selected users: [288 199]
Selected users: [391 477]
Selected users: [525  62]
Selected users: [202 555]
Selected users: [232 448]
Selected users: [610  89]
Selected users: [ 42 380]
Selected users: [305  18]
Selected users: [292  45]
Selected users: [514 202]
Selected users: [462 177]
Selected users: [339 275]
Selected users: [182 509]
Selected users: [534 313]
Selected users: [448  91]
Selected users: [42 66]
Selected users: [313 182]
Selected users: [105 597]
Selected users: [292 480]
Selected users: [103 560]
Selected users: 

0.7684584121642076

In [362]:
gsize3 = []
for i in range(1000):
    gsize3.append(group_pipeline(3))

mean(gsize3)

Selected users: [469  45 596]
Selected users: [202  42 381]
Selected users: [105  64 483]
Selected users: [509 226 249]
Selected users: [ 66 368 182]
Selected users: [ 19 368 509]
Selected users: [111 469 414]
Selected users: [ 19  62 313]
Selected users: [274  89 600]
Selected users: [292 232 298]
Selected users: [590 428 509]
Selected users: [597 561 313]
Selected users: [428 339 534]
Selected users: [ 18 525 199]
Selected users: [ 89   6 580]
Selected users: [ 19 555 318]
Selected users: [567 182 560]
Selected users: [597 448 555]
Selected users: [ 45 462 305]
Selected users: [ 42 318 448]
Selected users: [202 125 160]
Selected users: [382  45  51]
Selected users: [525  89 182]
Selected users: [599  28 380]
Selected users: [217  45 509]
Selected users: [ 42 226 105]
Selected users: [567 226 103]
Selected users: [ 18 534 484]
Selected users: [489 477  51]
Selected users: [560  57 132]
Selected users: [ 42 560 474]
Selected users: [448 387 484]
Selected users: [156 509  68]
Selected u

0.7521108740854099

In [363]:
gsize5 = []
for i in range(1000):
    gsize5.append(group_pipeline(5))

mean(gsize5)

Selected users: [ 64 111 489  45 156]
Selected users: [483 438 474 596 288]
Selected users: [ 62   6 125 525 339]
Selected users: [580 313  66 599 573]
Selected users: [177  68 288 298 596]
Selected users: [ 45  66 600 596  18]
Selected users: [573 105 202 606   6]
Selected users: [ 66 140 177 603 357]
Selected users: [509 469 182 596  89]
Selected users: [387 462 381 525 177]
Selected users: [599   6 177 414  89]
Selected users: [249 307  66 160 318]
Selected users: [ 21  18 480 474 132]
Selected users: [483 381  28 298 200]
Selected users: [ 21  64 448 477 200]
Selected users: [125 156 111  62 182]
Selected users: [105 292  28  51 573]
Selected users: [ 62 428 603 469  28]
Selected users: [ 42 560  19 156 603]
Selected users: [232 477 294 514 610]
Selected users: [ 28 597 219 202 517]
Selected users: [199 610 590 111 560]
Selected users: [182 125  89 514 226]
Selected users: [140   6 387 567 489]
Selected users: [232 606  21 387 200]
Selected users: [ 19 573 517  21 380]
Selected use

0.7304774174974252

In [364]:
#for 100 iterations the scores are 0.7982 0.7356 0.7274

In [ ]:
# for 1000 iterations the scores are 0.7684 0.7521 0.7304

In [371]:
from scipy.stats import ttest_ind

ttest_ind(gsize2, gsize3)

Ttest_indResult(statistic=2.4867191284976133, pvalue=0.012973808142828102)

In [372]:
ttest_ind(gsize2, gsize3)

Ttest_indResult(statistic=2.4867191284976133, pvalue=0.012973808142828102)

In [373]:
ttest_ind(gsize3, gsize5)

Ttest_indResult(statistic=3.567956610174897, pvalue=0.00036825450482157873)

We can see that the group size makes a significant difference regarding the performance of group recommendations